# Feature Selection

In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math
from implementations import *
from proj1_helpers import *
from misc_helpers import *
from plot_functions import *
from ml_math import *
%load_ext autoreload
%autoreload 2

In [ ]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
y_norm = normalize(y)
tX_norm = normalize(tX)

In [4]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

def cross_validation(y, x, k_fold, solver = 'LS',lambda_ = 0):
    """return the loss of ridge regression."""
    seed = 1
    k_indices = build_k_indices(y, k_fold, seed)
    
    mse_tr = 0
    mse_te = 0
    for k in range(k_fold):
        # get k'th subgroup in test, others in train:
        test_indices = k_indices[k]
        train_indices = np.delete(k_indices,k,0).flatten()
        x_tr = x[train_indices]
        y_tr = y[train_indices]
        x_te = x[test_indices]
        y_te = y[test_indices]

        # Least squares:
        if solver == 'LS':
            w, loss = least_squares(y_tr, x_tr)
        elif solver == 'RR':
            w, loss = ridge_regression(y_tr, x_tr, lambda_)
        else:
            raise('Error')

        # calculate the loss for train and test data: 
        loss_tr = compute_MSE(y_tr, x_tr, w)
        loss_te = compute_MSE(y_te, x_te, w)
    
        mse_tr += loss_tr/k_fold
        mse_te += loss_te/k_fold
        
    
    return mse_tr, mse_te, w

# Do some crazy feature selection here

In [8]:
def build_poly(x, degree, linear = False):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    if linear == False:
        D = len(x[0,:])
        N = len(x[:,0])
        new_x = np.ones((N,1)) #add bias
        if degree>=1:
            for i in range(1,degree+1):
                new_x = np.append(new_x,x**i,axis=1) 
        return new_x
    else:
        m = np.zeros((len(x),degree+1))
        for j in range(degree+1):
            m[:,j] = x**j
        return m

Here is the noise level. Everything that we add should beat this error

In [9]:
#only constant feature
tX0 = build_poly(tX,0)
print(cross_validation(y,tX0,4))
#only constant feature
tX0 = build_poly(tX_norm,0)
print(cross_validation(y,tX0,4))

(0.45049216308621154, 0.45050112681244814, array([-0.31568]))
(0.45049216308621154, 0.45050112681244814, array([-0.31568]))


In [10]:
#Test every degree for every feature
n,p = np.shape(tX)
loss = []
good_feat = []
min_loss = np.ones(p)*10000
min_deg = np.zeros(p)
for deg in [0,1,2,3,4,5,6,7,8,9,10]:
    for i in range(p):
        tX_ = build_poly(tX[:,i],deg, linear=True)
        loss_tr,loss_te,w = cross_validation(y,tX_,5)
        if loss_te<min_loss[i]:
            min_loss[i] = loss_te
            min_deg[i] = deg
        if loss_te < 0.45:
            good_feat.append((i,deg))


In [11]:
#all of these are good features
id_min_loss = np.where(min_loss < 0.43)
id_min_loss

(array([ 0,  1,  4,  5,  6,  9, 10, 11, 12, 13]),)

In [12]:
#concatenate all good feature in tX0
tX0 = build_poly(tX,0)
for i in  id_min_loss[0]:
    deg = int(min_deg[i])
    tX0 = np.append(tX0, build_poly(tX[:,i], deg, linear=True),1)
tX0

array([[ 1.00000000e+00,  1.00000000e+00,  1.38470000e+02, ...,
         1.06523904e+03,  3.47672719e+04,  1.13473422e+06],
       [ 1.00000000e+00,  1.00000000e+00,  1.60937000e+02, ...,
         1.76517620e+03,  7.41621127e+04,  3.11584700e+06],
       [ 1.00000000e+00,  1.00000000e+00, -9.99000000e+02, ...,
         1.03387972e+03,  3.32433684e+04,  1.06890727e+06],
       ...,
       [ 1.00000000e+00,  1.00000000e+00,  1.05457000e+02, ...,
         1.26992450e+03,  4.52550293e+04,  1.61270823e+06],
       [ 1.00000000e+00,  1.00000000e+00,  9.49510000e+01, ...,
         7.80867136e+02,  2.18205512e+04,  6.09753484e+05],
       [ 1.00000000e+00,  1.00000000e+00, -9.99000000e+02, ...,
         1.84925801e+03,  7.95236422e+04,  3.41975518e+06]])

In [13]:
lambdas = np.logspace(-10, 0, 30)
rmse_tr = []
rmse_te = []
mse_te_min = 10000
w0 = np.ones((p))
for lambda_ in lambdas:
    mse_tr, mse_te, w = cross_validation(y,tX0,5,solver = 'RR',lambda_ = lambda_)
    rmse_tr = np.append(rmse_tr,mse_tr)
    rmse_te = np.append(rmse_te,mse_te)
    if mse_te < mse_te_min:
        w0 = w
        mse_te_min = mse_te
        

# Test for AIcrowd

In [ ]:
DATA_TEST_PATH = 'data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
#tX_test = normalize(tX_test)
#concatenate all good feature in tX0
tX_test_0 = build_poly(tX_test,0)
for i in  id_min_loss[0]:
    deg = int(min_deg[i])
    tX_test_0 = np.append(tX_test_0, build_poly(tX_test[:,i], deg, linear=True),1)
np.shape(tX_test_0)

In [ ]:
w0

In [ ]:
OUTPUT_PATH = 'result/feature_selection.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w0, tX_test_0)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

# More advanced

Let's start from Remi's conclusion. Which is to take only the feature.\
1, 2, 4, 5, 7, 10, 11, 13, 14, 22, 23, 29, 30 (the id in feature is one less)\
First, delete where there is undefined feature. We will use:\
2, 4, 10, 11, 14, 22, 23, 30

Let's see for each, which of the degree is the best

In [ ]:
#Alogrithm who will add the feature with the degree only if it improves the model.
good_feat = range(30)
degrees = [0,1,2,3,4,5,6,7,8,9,10]
losses = np.zeros((len(degrees),len(good_feat)))
#initialize model
tX_model = build_poly(tX[:,1],1,linear ='True')
for feat in good_feat:
    loss = 10000
    for deg in degrees:
        tX0 = build_poly(tX[:,feat],deg,linear ='True')
        loss_tr, loss_te, w = cross_validation(y, tX0, 5, solver = 'LS')
        if loss_te > 0.45:
            loss_te = 0.45
        losses[deg,0] = deg
        losses[deg,1] = loss_te
    plt.figure(figsize=(14, 6))
    plt.plot(losses[:,0],losses[:,1])
    plt.title('Feature %i' %feat)
    plt.xticks(losses[:,0])
    
    plt.show()

The following algorithm add a feature, elevate to a power p, to the model if it improves the model

In [ ]:
#Alogrithm who will add the feature with the degree only if it improves the model.
good_feat = [1,3,9,10,13,21,22,29]
degrees = [1,2,3,4,5,6,1/2,1/3,1/4,1/5,1/6,\
          -1,-2,-3,-4,-5,-1/2,-1/3,-1/4,-1/5,-1/6]
#initialize model
tX_model = build_poly(tX[:,1],0,linear ='True')
w_model = np.ones(1)
model = np.zeros((1,2))
losses = np.array([])
losses_tr = np.array([])
for feat in good_feat:
    print(feat)
    loss = 10000
    for deg in degrees:
        tX_try = np.append(tX_model, np.array([tX_norm[:,feat]**deg]).T,axis=1)
        loss_tr, loss_te, w = cross_validation(y, tX_try, 5, solver = 'LS')
        if loss_te < loss:
            loss = loss_te
            tX_model = tX_try.copy()
            #save model
            model = np.append(model,np.array([[int(feat),deg]]),axis=0)
            losses = np.append(losses,loss)
            losses_tr = np.append(losses_tr,loss_tr)
            w_model = w
plt.figure(figsize=(14, 6))
plt.plot(losses)
plt.plot(losses_tr)
plt.title('losses')

plt.show()

#Save model
model_1 = model
loss_1 = losses[-1]
w_1 = w_model
tX_1 = tX_model

We can see that the loss decrease well. We should use this model

# With trigonometrie

As we see that it's good functionning, let's take all feature where it cannot be singular

In [ ]:
#Alogrithm who will add the feature with the degree only if it improves the model.
good_feat = [1,2,3,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,29]
degrees = [1,2,3,4,5,6,1/2,1/3,1/4,1/5,1/6,\
          -1,-2,-3,-4,-5,-6,-1/2,-1/3,-1/4,-1/5,-1/6]
#initialize model
tX_model = build_poly(tX[:,1],0,linear ='True')
w_model = np.ones(1)
model = np.zeros((1,2))
losses = np.array([])
for feat in good_feat:
    print(feat)
    loss = 10000
    # calucate a polynomial
    for deg in degrees:
        tX_try = np.append(tX_model, np.array([tX_norm[:,feat]**deg]).T,axis=1)
        loss_tr, loss_te, w = cross_validation(y, tX_try, 5, solver = 'LS')
        if loss_te < loss:
            loss = loss_te
            tX_model = tX_try.copy()
            #save model
            model = np.append(model,np.array([[int(feat),deg]]),axis=0)
            losses = np.append(losses,loss)
            w_model = w
    #calculate arctan
    tX_try = np.append(tX_model, np.array([np.arctan(tX_norm[:,feat])]).T,axis=1)
    loss_tr, loss_te, w = cross_validation(y, tX_try, 5, solver = 'LS')
    if loss_te < loss:
        loss = loss_te
        tX_model = tX_try.copy()
        #save model
        model = np.append(model,np.array([[int(feat),'arctan']]),axis=0)
        losses = np.append(losses,loss)
        w_model = w
    #calculate sinus
    tX_try = np.append(tX_model, np.array([np.sin(tX[:,feat])]).T,axis=1)
    loss_tr, loss_te, w = cross_validation(y, tX_try, 5, solver = 'LS')
    if loss_te < loss:
        loss = loss_te
        tX_model = tX_try.copy()
        #save model
        model = np.append(model,np.array([[int(feat),'sin']]),axis=0)
        losses = np.append(losses,loss)
        w_model = w
    #calculate cosinus
    tX_try = np.append(tX_model, np.array([np.cos(tX[:,feat])]).T,axis=1)
    loss_tr, loss_te, w = cross_validation(y, tX_try, 5, solver = 'LS')
    if loss_te < loss:
        loss = loss_te
        tX_model = tX_try.copy()
        #save model
        model = np.append(model,np.array([[int(feat),'cos']]),axis=0)
        losses = np.append(losses,loss)
        w_model = w              
            
plt.figure(figsize=(14, 6))
plt.plot(losses)
plt.title('losses')

plt.show()

model_2 = model
loss_2 = losses[-1]
w_2 = w_model
tX_2 = tX_model

The following takes all features advised by Rémi

In [ ]:
#Alogrithm who will add the feature with the degree only if it improves the model.
good_feat = [0, 1, 3, 4, 6, 9, 10, 12, 13, 21, 22, 28, 29]
degrees = [1,2,3,4,5,6,1/2,1/3,1/4,1/5,1/6,\
          -1,-2,-3,-4,-5,-6,-1/2,-1/3,-1/4,-1/5,-1/6]
#initialize model
tX_model = build_poly(tX[:,1],0,linear ='True')
w_model = np.ones(1)
model = np.zeros((1,2))
losses = np.array([])
for feat in good_feat:
    print(feat)
    loss = 10000
    # calucate a polynomial
    for deg in degrees:
        tX_try = np.append(tX_model, np.array([tX_norm[:,feat]**deg]).T,axis=1)
        loss_tr, loss_te, w = cross_validation(y, tX_try, 5, solver = 'LS')
        if loss_te < loss:
            loss = loss_te
            tX_model = tX_try.copy()
            #save model
            model = np.append(model,np.array([[int(feat),deg]]),axis=0)
            losses = np.append(losses,loss)
            w_model = w
    #calculate arctan
    tX_try = np.append(tX_model, np.array([np.arctan(tX_norm[:,feat])]).T,axis=1)
    loss_tr, loss_te, w = cross_validation(y, tX_try, 5, solver = 'LS')
    if loss_te < loss:
        loss = loss_te
        tX_model = tX_try.copy()
        #save model
        model = np.append(model,np.array([[int(feat),'arctan']]),axis=0)
        losses = np.append(losses,loss)
        w_model = w
    #calculate sinus
    tX_try = np.append(tX_model, np.array([np.sin(tX[:,feat])]).T,axis=1)
    loss_tr, loss_te, w = cross_validation(y, tX_try, 5, solver = 'LS')
    if loss_te < loss:
        loss = loss_te
        tX_model = tX_try.copy()
        #save model
        model = np.append(model,np.array([[int(feat),'sin']]),axis=0)
        losses = np.append(losses,loss)
        w_model = w
    #calculate cosinus
    tX_try = np.append(tX_model, np.array([np.cos(tX[:,feat])]).T,axis=1)
    loss_tr, loss_te, w = cross_validation(y, tX_try, 5, solver = 'LS')
    if loss_te < loss:
        loss = loss_te
        tX_model = tX_try.copy()
        #save model
        model = np.append(model,np.array([[int(feat),'sin']]),axis=0)
        losses = np.append(losses,loss)
        w_model = w              
            
plt.figure(figsize=(14, 6))
plt.plot(losses)
plt.title('losses')

plt.show()

model_3 = model
loss_3 = losses[-1]
w_3 = w_model
tX_3 = tX_model

Let's try all feature

In [ ]:
#Alogrithm who will add the feature with the degree only if it improves the model.
good_feat = range(30)
degrees = [1,2,3,4,5,6,1/2,1/3,1/4,\
          -1,-2,-3,-4,-5,-6,-1/2,-1/3,-1/4]
#initialize model
tX_model = build_poly(tX_norm[:,1],0,linear ='True')
w_model = np.ones(1)
model = np.zeros((1,2))
losses = np.array([])
for feat in good_feat:
    print(feat)
    loss = 10000
    # calucate a polynomial
    for deg in degrees:
        tX_try = np.append(tX_model, np.array([tX_norm[:,feat]**deg]).T,axis=1)
        loss_tr, loss_te, w = cross_validation(y, tX_try, 5, solver = 'LS')
        if loss_te < loss:
            loss = loss_te
            tX_model = tX_try.copy()
            #save model
            model = np.append(model,np.array([[int(feat),deg]]),axis=0)
            losses = np.append(losses,loss)
            w_model = w
    #calculate arctan
    tX_try = np.append(tX_model, np.array([np.arctan(tX_norm[:,feat])]).T,axis=1)
    loss_tr, loss_te, w = cross_validation(y, tX_try, 5, solver = 'LS')
    if loss_te < loss:
        loss = loss_te
        tX_model = tX_try.copy()
        #save model
        model = np.append(model,np.array([[int(feat),'arctan']]),axis=0)
        losses = np.append(losses,loss)
        w_model = w
    #calculate sinus
    tX_try = np.append(tX_model, np.array([np.sin(tX[:,feat])]).T,axis=1)
    loss_tr, loss_te, w = cross_validation(y, tX_try, 5, solver = 'LS')
    if loss_te < loss:
        loss = loss_te
        tX_model = tX_try.copy()
        #save model
        model = np.append(model,np.array([[int(feat),'sin']]),axis=0)
        losses = np.append(losses,loss)
        w_model = w
    #calculate cosinus
    tX_try = np.append(tX_model, np.array([np.cos(tX[:,feat])]).T,axis=1)
    loss_tr, loss_te, w = cross_validation(y, tX_try, 5, solver = 'LS')
    if loss_te < loss:
        loss = loss_te
        tX_model = tX_try.copy()
        #save model
        model = np.append(model,np.array([[int(feat),'cos']]),axis=0)
        losses = np.append(losses,loss)
        w_model = w              
            
plt.figure(figsize=(14, 6))
plt.plot(losses)
plt.title('losses')

plt.show()

model_4 = model
loss_4 = losses[-1]
w_4 = w_model
tX_4 = tX_model

# Rebuild model for another X to evaluate the model

Choose the best model

In [ ]:
print('loss_1: ', loss_1, 'shape :', np.shape(model_1))
print('loss_2: ', loss_2, 'shape :', np.shape(model_2))
print('loss_3: ', loss_3, 'shape :', np.shape(model_3))
print('loss_4: ', loss_4, 'shape :', np.shape(model_4))

In [ ]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
#choose the best model
model = model_3

#start with bias
tX_model = build_poly(tX[:,1],0,linear ='True')
tX_norm = normalize(tX)
#create model
for feat, deg in model:
    feat = int(float(feat.item()))
    print(feat,deg)
    if deg == 'arctan':
        tX_model = np.append(tX_model, np.array([np.arctan(tX_norm[:,feat])]).T,axis=1)
    elif deg == 'cos':
        tX_model = np.append(tX_model, np.array([np.cos(tX[:,feat])]).T,axis=1)
    elif deg =='sin':
        tX_model = np.append(tX_model, np.array([np.sin(tX[:,feat])]).T,axis=1)
    else :
        deg = float(deg)
        if deg != 0:
            tX_model = np.append(tX_model, np.array([tX_norm[:,feat]**deg]).T,axis=1)

#train weights
weights, loss = least_squares(y, tX_model)
print(loss)

In [ ]:
#Now Build the tX train data set

#start with bias
tX_model_test = build_poly(tX_test[:,1],0,linear ='True')
tX_test_norm = normalize(tX_test)
#create model
for feat, deg in model:
    feat = int(float(feat.item()))
    print(feat,deg)
    if deg == 'arctan':
        tX_model_test = np.append(tX_model_test, np.array([np.arctan(tX_test_norm[:,feat])]).T,axis=1)
    elif deg == 'cos':
        tX_model_test = np.append(tX_model_test, np.array([np.cos(tX_test[:,feat])]).T,axis=1)
    elif deg =='sin':
        tX_model_test = np.append(tX_model_test, np.array([np.sin(tX_test[:,feat])]).T,axis=1)
    else :
        deg = float(deg)
        if deg != 0:
            tX_model_test = np.append(tX_model_test, np.array([tX_test_norm[:,feat]**deg]).T,axis=1)

In [ ]:
OUTPUT_PATH = 'data/feature_selection_best_30.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_model_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)